# The Nurse Scheduling Problem


Imagine you are responsible for scheduling the shifts for the nurses in your hospital
department for this week. There are three shifts in a day – morning, afternoon, and night –
and for each shift, you need to assign one or more of the eight nurses that work in your
department. If this sounds like a simple task, take a look at the list of relevant hospital
rules:
- A nurse is not allowed to work two consecutive shifts.
- A nurse is not allowed to work more than five shifts per week.

The number of nurses per shift in your department should fall within the following limits:
- Morning shift: 2–3 nurses
- Afternoon shift: 2–4 nurses
- Night shift: 1–2 nurses

- In addition, each nurse can have shift preferences. For example, one nurse prefers to only
work morning shifts, another nurse prefers to not work afternoon shifts, and so on.
This task is an example of the nurse scheduling problem (NSP), which can have many
variants. Possible variations may include different specialties for different nurses, the
ability to work on cover shifts (overtime), or even different types of shifts – such as 8-hour
shifts and 12-hour shifts.
- By now, it probably looks like a good idea to write a program that will do the scheduling
for you. Why not apply our knowledge of genetic algorithms to implement such a
program? As usual, we will start by representing the solution to the problem





## Create The Nurse Scheduling Problem


Create a class called NurseSchedulingProblem that will represent the nurse scheduling problem with the following methods:

- `__init__(self, penality)`: Initializes the class with the given hardConstraintPenalty value. This value will be used to calculate the total cost of the violations in the schedule.

- The class uses the following method to convert the given schedule into a dictionary with a
separate schedule for each nurse:
  - getNurseShifts(schedule)

The following methods are used to count the various types of violations:

- countConsecutiveShiftViolations(nurseShiftsDict)
- countShiftsPerWeekViolations(nurseShiftsDict)
- countNursesPerShiftViolations(nurseShiftsDict)
- countShiftPreferenceViolations(nurseShiftsDict)


- `getCost(schedule)`: Calculates the total cost of the various violations in the given schedule. This method uses the value of the hardConstraintPenalty variable.
- `printScheduleInfo(schedule)`: Prints the schedule and violations details.



Should allow the following sample implementation
```python
nurses = NurseSchedulingProblem(10)

randomSolution = np.random.randint(2, size=len(nurses))
print("Random Solution = ")
print(randomSolution)
print()

nurses.printScheduleInfo(randomSolution)

print("Total Cost = ", nurses.getCost(randomSolution))

```

```
Schedule for each nurse:
A : [0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 0]
B : [0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0]
C : [0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0]
D : [0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0]
E : [1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1]
F : [1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1]
G : [0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 1 1 1]
H : [1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0]
consecutive shift violations =  39

weekly Shifts =  [12, 8, 9, 10, 8, 15, 10, 7]
Shifts Per Week Violations =  39

Nurses Per Shift =  [3, 5, 4, 5, 3, 3, 5, 3, 3, 4, 3, 3, 5, 3, 3, 4, 3, 4, 5, 5, 3]
Nurses Per Shift Violations =  21

Shift Preference Violations =  30

Total Cost =  1020
```

### Bas Code


In [1]:
import numpy as np


class NurseSchedulingProblem:
    """This class encapsulates the Nurse Scheduling problem
    """

    def __init__(self, hardConstraintPenalty):
        """
        :param hardConstraintPenalty: the penalty factor for a hard-constraint violation
        """
        self.hardConstraintPenalty = hardConstraintPenalty

        # list of nurses:
        self.nurses = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

        # nurses' respective shift preferences - morning, evening, night:
        self.shiftPreference = [[1, 0, 0], [1, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 1, 1], [0, 1, 1], [1, 1, 1]]

        # min and max number of nurses allowed for each shift - morning, evening, night:
        self.shiftMin = [2, 2, 1]
        self.shiftMax = [3, 4, 2]

        # max shifts per week allowed for each nurse
        self.maxShiftsPerWeek = 5

        # number of weeks we create a schedule for:
        self.weeks = 1

        # useful values:
        self.shiftPerDay = len(self.shiftMin)
        self.shiftsPerWeek = 7 * self.shiftPerDay

    def __len__(self):
        """
        :return: the number of shifts in the schedule
        """
        return len(self.nurses) * self.shiftsPerWeek * self.weeks


    def getCost(self, schedule):
        """
        Calculates the total cost of the various violations in the given schedule
        ...
        :param schedule: a list of binary values describing the given schedule
        :return: the calculated cost
        """

        if len(schedule) != self.__len__():
            raise ValueError("size of schedule list should be equal to ", self.__len__())

        # convert entire schedule into a dictionary with a separate schedule for each nurse:
        nurseShiftsDict = self.getNurseShifts(schedule)

        # count the various violations:
        consecutiveShiftViolations = self.countConsecutiveShiftViolations(nurseShiftsDict)
        shiftsPerWeekViolations = self.countShiftsPerWeekViolations(nurseShiftsDict)[1]
        nursesPerShiftViolations = self.countNursesPerShiftViolations(nurseShiftsDict)[1]
        shiftPreferenceViolations = self.countShiftPreferenceViolations(nurseShiftsDict)

        # calculate the cost of the violations:
        hardContstraintViolations = consecutiveShiftViolations + nursesPerShiftViolations + shiftsPerWeekViolations
        softContstraintViolations = shiftPreferenceViolations

        return self.hardConstraintPenalty * hardContstraintViolations + softContstraintViolations

    def getNurseShifts(self, schedule) -> dict[str: list[int]]:
        """
        Converts the entire schedule into a dictionary with a separate schedule for each nurse
        :param schedule: a list of binary values describing the given schedule
        :return: a dictionary with each nurse as a key and the corresponding shifts as the value
        """
        shiftsPerNurse = self.__len__() // len(self.nurses)
        nurseShiftsDict = {}
        shiftIndex = 0

        for nurse in self.nurses:
            nurseShiftsDict[nurse] = schedule[shiftIndex:shiftIndex + shiftsPerNurse]
            shiftIndex += shiftsPerNurse

        return nurseShiftsDict

    def countConsecutiveShiftViolations(self, nurseShiftsDict) -> int:
        """
        Counts the consecutive shift violations in the schedule
        :param nurseShiftsDict: a dictionary with a separate schedule for each nurse
        :return: count of violations found
        """
        violations = 0
        # TODO iterate over the shifts of each nurse:
        
        return violations

    def countShiftsPerWeekViolations(self, nurseShiftsDict) -> set[int:int]:
        """
        Counts the max-shifts-per-week violations in the schedule
        :param nurseShiftsDict: a dictionary with a separate schedule for each nurse
        :return: count of violations found
        """
        violations = 0
        weeklyShiftsList = []
        # TODO iterate over the shifts of each nurse:

        return weeklyShiftsList, violations

    def countNursesPerShiftViolations(self, nurseShiftsDict) -> set[int:int]:
        """
        Counts the number-of-nurses-per-shift violations in the schedule
        :param nurseShiftsDict: a dictionary with a separate schedule for each nurse
        :return: count of violations found
        """
        # sum the shifts over all nurses:
        totalPerShiftList = [sum(shift) for shift in zip(*nurseShiftsDict.values())]

        violations = 0
        # TODO iterate over all shifts and count violations:

        return totalPerShiftList, violations

    def countShiftPreferenceViolations(self, nurseShiftsDict) -> int:
        """
        Counts the nurse-preferences violations in the schedule
        :param nurseShiftsDict: a dictionary with a separate schedule for each nurse
        :return: count of violations found
        """
        violations = 0
        # TOOD Complete
        
        return violations

    def printScheduleInfo(self, schedule):
        """
        Prints the schedule and violations details
        :param schedule: a list of binary values describing the given schedule
        """
        pass

In [2]:
nurses = NurseSchedulingProblem(10)

randomSolution = np.random.randint(2, size=len(nurses))
print("Random Solution = ")
print(randomSolution)
print()

nurses.printScheduleInfo(randomSolution)

print("Total Cost = ", nurses.getCost(randomSolution))


Random Solution = 
[1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1
 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1
 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0
 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1
 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0]

Schedule for each nurse:
A : [1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1]
B : [1 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0]
C : [0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 1 1 0 1 0]
D : [1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0]
E : [1 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1]
F : [0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1]
G : [0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0]
H : [1 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0]
consecutive shift violations =  35

weekly Shifts =  [12, 10, 12, 12, 11, 7, 7, 10]
Shifts Per Week Violations =  41

Nurses Per Shift =  [5, 4, 4, 2, 4, 1, 2, 6, 5, 5, 4, 1, 4, 5, 2, 6, 3, 5, 4, 6, 3]
Nurses Per Shift Violations